In [1]:
# Set up
import sys
import numpy as np
import pandas as pd
from tqdm.autonotebook import tqdm

import torch
import torch.nn as nn
from torch.nn import functional as F
from transformers import ViTFeatureExtractor, ViTForImageClassification

import warnings
warnings.filterwarnings('ignore')

sys.path.append('..')

dataset_path = '../Datasets/CIFAR10'
img_path = dataset_path + '/images'
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [2]:
# Set data owner id of interest
data_owner_id = 'B'

In [3]:
def seed_everything(seed=20):
    """set seed for all"""
    import os
    import torch
    import random
    import numpy as np
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
seed_everything()

In [4]:
def get_vit_model(device='cpu'):
    # download vision transformer model 
    vit_feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224-in21k')
    vit_model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224', output_hidden_states=True).to(device)

    # freeze the pre-trained model
    vit_model.eval()
    for param in vit_model.parameters():
        param.requires_grad = False
    return vit_feature_extractor, vit_model

In [5]:
# Training hyperparameters
num_epochs = 10
learning_rate = 0.001

In [6]:
class CNNClassifier(nn.Module):
    def __init__(self, in_dim, out_dim, dropout=0.1):
        super(CNNClassifier, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 32, (4, 4), stride=2, padding=0),
            nn.ReLU(),
            nn.LazyBatchNorm2d()
        )
        self.conv2 = nn.Sequential(
            nn.LazyConv2d(64, (2, 2), stride=1, padding=0),
            nn.ReLU(),
            nn.LazyBatchNorm2d()
        )

        self.conv3 = nn.Sequential(
            nn.LazyConv2d(64, (1, 1), stride=1),
            nn.ReLU()
        )

        self.fc_out = nn.Sequential(
            nn.Flatten(),
            nn.LazyLinear(256),
            nn.ReLU(),
            nn.LazyLinear(out_dim)
        )

        self.dropout = nn.Dropout(p=dropout)

    def forward(self, x):
        x = self.conv3(self.conv2(self.conv1(x)))
        x = self.dropout(x)
        x = self.fc_out(x)
        return x


In [7]:
# Load data owner dataset
data_owner_dataset = pd.read_excel(dataset_path + '/CIFAR10dataOwnerInfo.xlsx', sheet_name=data_owner_id)
data_owner_dataset.image = [f'{img_path}/{image}' for image in data_owner_dataset.image]

# Create data owner's model 
num_classes = data_owner_dataset.label_name.nunique()
model = CNNClassifier(in_dim=(3,32,32), out_dim=num_classes, dropout=0.1).to(device)

# Initialize loss function (criterion) and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [8]:
# Label encoding and get images and labels as lists
def label_enc(data_owner_dataset):
    from sklearn.preprocessing import LabelEncoder
    le = LabelEncoder().fit(data_owner_dataset.label_name)
    label2id = {k:v for k, v in zip(le.classes_, le.transform(le.classes_))}
    id2label = {v:k for k, v in label2id.items()}
    labels = le.transform(data_owner_dataset.label_name)
    images = data_owner_dataset.image.tolist()
    return images, labels, label2id, id2label

images, labels, label2id, id2label = label_enc(data_owner_dataset)

In [9]:
from core.ai.dataset import get_loader
import albumentations as A
import albumentations.pytorch.transforms as T
train_transform = A.Compose([
    A.Resize(224,224),
    A.Normalize(mean=[0.4914, 0.4822, 0.4465],
                std=[0.2470, 0.2435, 0.2616], 
                max_pixel_value=1),
    A.Flip(p=0.5),
    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
    T.ToTensorV2()
])

data_loader = get_loader(images, labels, None, train_transform,
                         pre_trained_model=None, device=device)


In [10]:
# Test sample input for the model
sample_batch = next(iter(data_loader))
sample_images, sample_labels = sample_batch
logits = model(sample_images.to(device))
print(model)
print('Batch image shape:', list(sample_images.shape))
print('Batch label shape:', list(sample_labels.shape))
print('Model output shape:', list(logits.shape))

CNNClassifier(
  (conv1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(4, 4), stride=(2, 2))
    (1): ReLU()
    (2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(2, 2), stride=(1, 1))
    (1): ReLU()
    (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): Sequential(
    (0): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
    (1): ReLU()
  )
  (fc_out): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=774400, out_features=256, bias=True)
    (2): ReLU()
    (3): Linear(in_features=256, out_features=3, bias=True)
  )
  (dropout): Dropout(p=0.1, inplace=False)
)
Batch image shape: [32, 3, 224, 224]
Batch label shape: [32]
Model output shape: [32, 3]


In [11]:
model.train()
for epoch in range(num_epochs):
    epoch_loss_list = []
    epoch_acc_sum = [0, 0]
    for batch_images, batch_labels in tqdm(data_loader):
        logits = model(batch_images.to(device))
        batch_labels = batch_labels.long().to(device)
        optimizer.zero_grad()
        loss = criterion(logits, batch_labels)
        loss.backward()
        optimizer.step()
        
        epoch_loss_list.append(loss.item())
        epoch_acc_sum[0] += (logits.argmax(1) == batch_labels).sum().item()
        epoch_acc_sum[1] += len(batch_labels)
    
    print(f'[ {epoch:2d}:{num_epochs} ]\tloss={np.mean(epoch_loss_list):.3f}, \
          acc={epoch_acc_sum[0]}/{epoch_acc_sum[1]}={epoch_acc_sum[0]/epoch_acc_sum[1]:.3f}')


  0%|          | 0/185 [00:14<?, ?it/s]


RuntimeError: CUDA out of memory. Tried to allocate 758.00 MiB (GPU 0; 4.00 GiB total capacity; 2.95 GiB already allocated; 0 bytes free; 2.96 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF